# 인페인팅

인페인팅은 이미지의 특정 영역을 교체하거나 편집합니다. 이는 결함 및 아티팩트 제거와 같은 이미지 복원 또는 이미지 영역을 완전히 새로운 것으로 교체하는 데 유용한 도구입니다. 인페인팅은 이미지의 어떤 영역을 채울지 결정하기 위해 마스크에 의존합니다. 인페인트할 영역은 흰색 픽셀로 표시되고 유지할 영역은 검은색 픽셀로 표시됩니다. 흰색 픽셀은 프롬프트에 의해 채워집니다.

🤗 Diffusers를 사용하면 다음과 같이 인페인팅을 수행할 수 있습니다.

1. [AutoPipelineForInpainting](https://huggingface.co/docs/diffusers/main/en/api/pipelines/auto_pipeline#diffusers.AutoPipelineForInpainting) 클래스를 사용하여 인페인팅 체크포인트를 로드합니다. 이렇게 하면 체크포인트를 기반으로 로드할 적절한 파이프라인 클래스가 자동으로 감지됩니다.

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForInpainting.from_pretrained(
    "kandinsky-community/kandinsky-2-2-decoder-inpaint", torch_dtype=torch.float16
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

<Tip>

가이드 전체에서 메모리를 절약하고 추론 속도를 높이기 위해 [enable_model_cpu_offload()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.enable_model_cpu_offload) 및 [enable_xformers_memory_efficient_attention()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.enable_xformers_memory_efficient_attention)을 사용하는 것을 알 수 있습니다. PyTorch 2.0을 사용하는 경우 파이프라인에서 [enable_xformers_memory_efficient_attention()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.enable_xformers_memory_efficient_attention)을 호출할 필요가 없습니다. 이미 PyTorch 2.0의 기본 [스케일드 닷 프로덕트 어텐션](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/fp16#scaled-dot-product-attention)을 사용하기 때문입니다.

</Tip>

2. 기본 이미지와 마스크 이미지를 로드합니다.

In [ ]:
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png")

3. 이미지를 인페인트할 프롬프트를 만들고 기본 이미지 및 마스크 이미지와 함께 파이프라인에 전달합니다.

In [ ]:
prompt = "a black cat with glowing eyes, cute, adorable, disney, pixar, highly detailed, 8k"
negative_prompt = "bad anatomy, deformed, ugly, disfigured"
image = pipeline(prompt=prompt, negative_prompt=negative_prompt, image=init_image, mask_image=mask_image).images[0]
make_image_grid([init_image, mask_image, image], rows=1, cols=3)

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">기본 이미지</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">마스크 이미지</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-cat.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">생성된 이미지</figcaption>
  </div>
</div>

## 마스크 이미지 만들기

이 가이드 전체에서 마스크 이미지는 편의를 위해 모든 코드 예제에 제공됩니다. 자신의 이미지에 인페인팅할 수 있지만 마스크 이미지를 만들어야 합니다. 아래 공간을 사용하여 마스크 이미지를 쉽게 만들 수 있습니다.

인페인트할 기본 이미지를 업로드하고 스케치 도구를 사용하여 마스크를 그립니다. 완료되면 **실행**을 클릭하여 마스크 이미지를 생성하고 다운로드합니다.

<iframe
  src="https://stevhliu-inpaint-mask-maker.hf.space"
  frameborder="0"
  width="850"
  height="450"
></iframe>

### 마스크 흐림 효과

`~VaeImageProcessor.blur` 메서드는 원본 이미지와 인페인트 영역을 혼합하는 방법에 대한 옵션을 제공합니다. 흐림 효과의 양은 `blur_factor` 매개변수에 의해 결정됩니다. `blur_factor`를 늘리면 마스크 가장자리에 적용되는 흐림 효과의 양이 증가하여 원본 이미지와 인페인트 영역 간의 전환이 부드러워집니다. `blur_factor`가 낮거나 0이면 마스크의 더 날카로운 가장자리가 유지됩니다.

이를 사용하려면 이미지 프로세서로 흐릿한 마스크를 만듭니다.

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image
from PIL import Image

pipeline = AutoPipelineForInpainting.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16).to('cuda')

mask = load_image("https://huggingface.co/datasets/YiYiXu/testing-images/resolve/main/seashore_mask.png")
blurred_mask = pipeline.mask_processor.blur(mask, blur_factor=33)
blurred_mask

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/YiYiXu/testing-images/resolve/main/seashore_mask.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">흐림 효과 없는 마스크</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/mask_blurred.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">흐림 효과 적용된 마스크</figcaption>
  </div>
</div>

## 인기 모델

[Stable Diffusion Inpainting](https://huggingface.co/runwayml/stable-diffusion-inpainting), [Stable Diffusion XL (SDXL) Inpainting](https://huggingface.co/diffusers/stable-diffusion-xl-1.0-inpainting-0.1) 및 [Kandinsky 2.2 Inpainting](https://huggingface.co/kandinsky-community/kandinsky-2-2-decoder-inpaint)은 인페인팅에 가장 인기 있는 모델 중 일부입니다. SDXL은 일반적으로 Stable Diffusion v1.5보다 고해상도 이미지를 생성하며 Kandinsky 2.2도 고품질 이미지를 생성할 수 있습니다.

### 스테이블 디퓨전 인페인팅

Stable Diffusion Inpainting은 인페인팅에 대해 512x512 이미지에서 미세 조정된 잠재 확산 모델입니다. 비교적 빠르고 좋은 품질의 이미지를 생성하므로 좋은 출발점입니다. 이 모델을 인페인팅에 사용하려면 프롬프트, 기본 이미지 및 마스크 이미지를 파이프라인에 전달해야 합니다.

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16, variant="fp16"
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 기본 이미지와 마스크 이미지 로드
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png")

generator = torch.Generator("cuda").manual_seed(92)
prompt = "concept art digital painting of an elven castle, inspired by lord of the rings, highly detailed, 8k"
image = pipeline(prompt=prompt, image=init_image, mask_image=mask_image, generator=generator).images[0]
make_image_grid([init_image, mask_image, image], rows=1, cols=3)

### 스테이블 디퓨전 XL (SDXL) 인페인팅

SDXL은 Stable Diffusion v1.5의 더 크고 강력한 버전입니다. 이 모델은 2단계 모델 프로세스를 따를 수 있습니다(각 모델은 단독으로 사용할 수도 있음). 기본 모델은 이미지를 생성하고 리파이너 모델은 해당 이미지를 가져와 세부 정보와 품질을 더욱 향상시킵니다. SDXL 사용 방법 및 매개변수 구성에 대한 자세한 내용은 [SDXL](https://huggingface.co/docs/diffusers/main/en/using-diffusers/sdxl) 가이드를 참조하십시오.

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForInpainting.from_pretrained(
    "diffusers/stable-diffusion-xl-1.0-inpainting-0.1", torch_dtype=torch.float16, variant="fp16"
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 기본 이미지와 마스크 이미지 로드
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png")

generator = torch.Generator("cuda").manual_seed(92)
prompt = "concept art digital painting of an elven castle, inspired by lord of the rings, highly detailed, 8k"
image = pipeline(prompt=prompt, image=init_image, mask_image=mask_image, generator=generator).images[0]
make_image_grid([init_image, mask_image, image], rows=1, cols=3)

### 칸딘스키 2.2 인페인팅

Kandinsky 모델 계열은 두 가지 모델을 사용한다는 점에서 SDXL과 유사합니다. 이미지 사전 모델은 이미지 임베딩을 생성하고 확산 모델은 이를 사용하여 이미지를 생성합니다. 이미지 사전 모델과 확산 모델을 별도로 로드할 수 있지만 Kandinsky 2.2를 사용하는 가장 쉬운 방법은 [AutoPipelineForInpainting](https://huggingface.co/docs/diffusers/main/en/api/pipelines/auto_pipeline#diffusers.AutoPipelineForInpainting) 클래스에 로드하는 것입니다. 이 클래스는 내부적으로 [KandinskyV22InpaintCombinedPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/kandinsky_v22#diffusers.KandinskyV22InpaintCombinedPipeline)을 사용합니다.

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForInpainting.from_pretrained(
    "kandinsky-community/kandinsky-2-2-decoder-inpaint", torch_dtype=torch.float16
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 기본 이미지와 마스크 이미지 로드
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png")

generator = torch.Generator("cuda").manual_seed(92)
prompt = "concept art digital painting of an elven castle, inspired by lord of the rings, highly detailed, 8k"
image = pipeline(prompt=prompt, image=init_image, mask_image=mask_image, generator=generator).images[0]
make_image_grid([init_image, mask_image, image], rows=1, cols=3)

<div class="flex flex-row gap-4">
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">기본 이미지</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-sdv1.5.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">스테이블 디퓨전 인페인팅</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-sdxl.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">스테이블 디퓨전 XL 인페인팅</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-kandinsky.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">칸딘스키 2.2 인페인팅</figcaption>
  </div>
</div>

## 인페인트 전용이 아닌 체크포인트

지금까지 이 가이드에서는 [stable-diffusion-v1-5/stable-diffusion-inpainting](https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-inpainting)과 같은 인페인트 전용 체크포인트를 사용했습니다. 하지만 [stable-diffusion-v1-5/stable-diffusion-v1-5](https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5)와 같은 일반 체크포인트도 사용할 수 있습니다. 두 체크포인트의 결과를 비교해 보겠습니다.

왼쪽 이미지는 일반 체크포인트에서 생성된 것이고 오른쪽 이미지는 인페인트 체크포인트에서 생성된 것입니다. 왼쪽 이미지는 그다지 깨끗하지 않고 모델이 인페인트해야 하는 영역의 윤곽선이 여전히 보이는 것을 즉시 알 수 있습니다. 오른쪽 이미지는 훨씬 깨끗하고 인페인트된 영역이 더 자연스러워 보입니다.

<hfoptions id="regular-specific">
<hfoption id="stable-diffusion-v1-5/stable-diffusion-v1-5">

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForInpainting.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16"
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 기본 이미지와 마스크 이미지 로드
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png")

generator = torch.Generator("cuda").manual_seed(92)
prompt = "concept art digital painting of an elven castle, inspired by lord of the rings, highly detailed, 8k"
image = pipeline(prompt=prompt, image=init_image, mask_image=mask_image, generator=generator).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

</hfoption>
<hfoption id="runwayml/stable-diffusion-inpainting">

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16, variant="fp16"
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 기본 이미지와 마스크 이미지 로드
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png")

generator = torch.Generator("cuda").manual_seed(92)
prompt = "concept art digital painting of an elven castle, inspired by lord of the rings, highly detailed, 8k"
image = pipeline(prompt=prompt, image=init_image, mask_image=mask_image, generator=generator).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

</hfoption>
</hfoptions>

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/non-inpaint-specific.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">stable-diffusion-v1-5/stable-diffusion-v1-5</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-specific.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">runwayml/stable-diffusion-inpainting</figcaption>
  </div>
</div>

그러나 이미지에서 개체를 지우는 것과 같은 기본적인 작업(예: 도로의 바위)의 경우 일반 체크포인트도 꽤 좋은 결과를 생성합니다. 일반 체크포인트와 인페인트 체크포인트 간에 눈에 띄는 차이는 없습니다.

<hfoptions id="inpaint">
<hfoption id="stable-diffusion-v1-5/stable-diffusion-v1-5">

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForInpainting.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16"
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 기본 이미지와 마스크 이미지 로드
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/road-mask.png")

image = pipeline(prompt="road", image=init_image, mask_image=mask_image).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

</hfoption>
<hfoption id="runwayml/stable-diffusion-inpaint">

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16, variant="fp16"
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 기본 이미지와 마스크 이미지 로드
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/road-mask.png")

image = pipeline(prompt="road", image=init_image, mask_image=mask_image).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

</hfoption>
</hfoptions>

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/regular-inpaint-basic.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">stable-diffusion-v1-5/stable-diffusion-v1-5</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/specific-inpaint-basic.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">runwayml/stable-diffusion-inpainting</figcaption>
  </div>
</div>

인페인트 전용이 아닌 체크포인트를 사용하는 단점은 전반적인 이미지 품질이 낮을 수 있지만 일반적으로 마스크 영역을 보존하는 경향이 있다는 것입니다(그래서 마스크 윤곽선이 보입니다). 인페인트 전용 체크포인트는 의도적으로 고품질 인페인트 이미지를 생성하도록 학습되었으며, 여기에는 마스크된 영역과 마스크되지 않은 영역 간의 보다 자연스러운 전환을 만드는 것이 포함됩니다. 결과적으로 이러한 체크포인트는 마스크되지 않은 영역을 변경할 가능성이 더 큽니다.

마스크되지 않은 영역을 보존하는 것이 작업에 중요한 경우 `VaeImageProcessor.apply_overlay` 메서드를 사용하여 이미지의 마스크되지 않은 영역을 동일하게 유지하도록 강제할 수 있지만 마스크된 영역과 마스크되지 않은 영역 간의 전환이 다소 부자연스러워질 수 있습니다.

In [ ]:
import PIL
import numpy as np
import torch

from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

device = "cuda"
pipeline = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    torch_dtype=torch.float16,
    variant="fp16"
)
pipeline = pipeline.to(device)

img_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png"
mask_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo_mask.png"

init_image = load_image(img_url).resize((512, 512))
mask_image = load_image(mask_url).resize((512, 512))

prompt = "Face of a yellow cat, high resolution, sitting on a park bench"
repainted_image = pipeline(prompt=prompt, image=init_image, mask_image=mask_image).images[0]
repainted_image.save("repainted_image.png")

unmasked_unchanged_image = pipeline.image_processor.apply_overlay(mask_image, init_image, repainted_image)
unmasked_unchanged_image.save("force_unmasked_unchanged.png")
make_image_grid([init_image, mask_image, repainted_image, unmasked_unchanged_image], rows=2, cols=2)

## 파이프라인 매개변수 구성

이미지 특징(품질 및 "창의성" 등)은 파이프라인 매개변수에 따라 달라집니다. 원하는 결과를 얻으려면 이러한 매개변수가 수행하는 작업을 아는 것이 중요합니다. 가장 중요한 매개변수를 살펴보고 변경 사항이 출력에 미치는 영향을 살펴보겠습니다.

### 강도

`strength`는 기본 이미지에 추가되는 노이즈의 양을 측정하는 것으로, 출력이 기본 이미지와 얼마나 유사한지에 영향을 줍니다.

* 📈 `strength` 값이 높으면 이미지에 더 많은 노이즈가 추가되고 노이즈 제거 프로세스가 더 오래 걸리지만 기본 이미지와 더 다른 고품질 이미지를 얻을 수 있습니다.
* 📉 `strength` 값이 낮으면 이미지에 더 적은 노이즈가 추가되고 노이즈 제거 프로세스가 더 빠르지만 이미지 품질이 그다지 좋지 않을 수 있으며 생성된 이미지가 기본 이미지와 더 유사합니다.

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16, variant="fp16"
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 기본 이미지와 마스크 이미지 로드
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png")

prompt = "concept art digital painting of an elven castle, inspired by lord of the rings, highly detailed, 8k"
image = pipeline(prompt=prompt, image=init_image, mask_image=mask_image, strength=0.6).images[0]
make_image_grid([init_image, mask_image, image], rows=1, cols=3)

<div class="flex flex-row gap-4">
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-strength-0.6.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">strength = 0.6</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-strength-0.8.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">strength = 0.8</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-strength-1.0.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">strength = 1.0</figcaption>
  </div>
</div>

### 안내 척도

`guidance_scale`은 텍스트 프롬프트와 생성된 이미지가 얼마나 정렬되는지에 영향을 줍니다.

* 📈 `guidance_scale` 값이 높으면 프롬프트와 생성된 이미지가 밀접하게 정렬되어 출력이 프롬프트의 더 엄격한 해석이 됩니다.
* 📉 `guidance_scale` 값이 낮으면 프롬프트와 생성된 이미지가 더 느슨하게 정렬되어 출력이 프롬프트와 더 다양할 수 있습니다.

`strength`와 `guidance_scale`을 함께 사용하여 모델의 표현력을 더욱 효과적으로 제어할 수 있습니다. 예를 들어, `strength`와 `guidance_scale` 값을 높게 조합하면 모델에 가장 창의적인 자유를 줄 수 있습니다.

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16, variant="fp16"
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 기본 이미지와 마스크 이미지 로드
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png")

prompt = "concept art digital painting of an elven castle, inspired by lord of the rings, highly detailed, 8k"
image = pipeline(prompt=prompt, image=init_image, mask_image=mask_image, guidance_scale=2.5).images[0]
make_image_grid([init_image, mask_image, image], rows=1, cols=3)

<div class="flex flex-row gap-4">
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-guidance-2.5.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">guidance_scale = 2.5</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-guidance-7.5.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">guidance_scale = 7.5</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-guidance-12.5.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">guidance_scale = 12.5</figcaption>
  </div>
</div>

### 부정적 프롬프트

부정적 프롬프트는 프롬프트와 반대 역할을 합니다. 즉, 모델이 이미지에 특정 항목을 생성하지 않도록 안내합니다. 이는 이미지 품질을 빠르게 향상시키고 모델이 원하지 않는 항목을 생성하는 것을 방지하는 데 유용합니다.

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16, variant="fp16"
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 기본 이미지와 마스크 이미지 로드
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png")

prompt = "concept art digital painting of an elven castle, inspired by lord of the rings, highly detailed, 8k"
negative_prompt = "bad architecture, unstable, poor details, blurry"
image = pipeline(prompt=prompt, negative_prompt=negative_prompt, image=init_image, mask_image=mask_image).images[0]
make_image_grid([init_image, mask_image, image], rows=1, cols=3)

<div class="flex justify-center">
  <figure>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-negative.png" />
    <figcaption class="text-center">negative_prompt = "bad architecture, unstable, poor details, blurry"</figcaption>
  </figure>
</div>

### 패딩 마스크 자르기

인페인팅 이미지 품질을 높이는 한 가지 방법은 [`padding_mask_crop`](https://huggingface.co/docs/diffusers/v0.25.0/en/api/pipelines/stable_diffusion/inpaint#diffusers.StableDiffusionInpaintPipeline.__call__.padding_mask_crop) 매개변수를 사용하는 것입니다. 이 옵션을 활성화하면 사용자가 지정한 패딩으로 마스크된 영역을 자르고 원본 이미지에서도 동일한 영역을 자릅니다. 이미지와 마스크 모두 인페인팅을 위해 더 높은 해상도로 업스케일링된 다음 원본 이미지에 오버레이됩니다. 이는 [StableDiffusionUpscalePipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/upscale#diffusers.StableDiffusionUpscalePipeline)과 같은 별도의 파이프라인을 사용하지 않고도 이미지 품질을 향상시키는 빠르고 쉬운 방법입니다.

파이프라인 호출에 `padding_mask_crop` 매개변수를 추가하고 원하는 패딩 값으로 설정합니다.

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image
from PIL import Image

generator = torch.Generator(device='cuda').manual_seed(0)
pipeline = AutoPipelineForInpainting.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16).to('cuda')

base = load_image("https://huggingface.co/datasets/YiYiXu/testing-images/resolve/main/seashore.png")
mask = load_image("https://huggingface.co/datasets/YiYiXu/testing-images/resolve/main/seashore_mask.png")

image = pipeline("boat", image=base, mask_image=mask, strength=0.75, generator=generator, padding_mask_crop=32).images[0]
image

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/baseline_inpaint.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">기본 인페인트 이미지</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/padding_mask_crop_inpaint.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">`padding_mask_crop`이 활성화된 인페인트 이미지</figcaption>
  </div>
</div>

## 연쇄된 인페인팅 파이프라인

[AutoPipelineForInpainting](https://huggingface.co/docs/diffusers/main/en/api/pipelines/auto_pipeline#diffusers.AutoPipelineForInpainting)은 다른 🤗 Diffusers 파이프라인과 연결하여 출력을 편집할 수 있습니다. 이는 다른 확산 파이프라인의 출력 품질을 향상시키는 데 종종 유용하며, 여러 파이프라인을 사용하는 경우 출력을 잠재 공간에 유지하고 동일한 파이프라인 구성 요소를 재사용하여 메모리 효율성을 높일 수 있습니다.

### 텍스트-이미지-인페인트

텍스트-이미지 및 인페인팅 파이프라인을 연결하면 생성된 이미지를 인페인트할 수 있으며, 처음부터 기본 이미지를 제공할 필요가 없습니다. 이렇게 하면 완전히 새로운 이미지를 생성할 필요 없이 좋아하는 텍스트-이미지 출력을 편리하게 편집할 수 있습니다.

텍스트-이미지 파이프라인으로 성을 만들어 시작합니다.

In [ ]:
import torch
from diffusers import AutoPipelineForText2Image, AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForText2Image.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

text2image = pipeline("concept art digital painting of an elven castle, inspired by lord of the rings, highly detailed, 8k").images[0]

위 출력의 마스크 이미지를 로드합니다.

In [ ]:
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_text-chain-mask.png")

그리고 마스크된 영역을 폭포로 인페인트해 보겠습니다.

In [ ]:
pipeline = AutoPipelineForInpainting.from_pretrained(
    "kandinsky-community/kandinsky-2-2-decoder-inpaint", torch_dtype=torch.float16
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

prompt = "digital painting of a fantasy waterfall, cloudy"
image = pipeline(prompt=prompt, image=text2image, mask_image=mask_image).images[0]
make_image_grid([text2image, mask_image, image], rows=1, cols=3)

<div class="flex flex-row gap-4">
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-text-chain.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">텍스트-이미지 변환</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-text-chain-out.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">인페인트</figcaption>
  </div>
</div>

### 인페인트-이미지-이미지

이미지-이미지 또는 업스케일러와 같은 다른 파이프라인 전에 인페인팅 파이프라인을 연결하여 품질을 향상시킬 수도 있습니다.

이미지를 인페인팅하여 시작합니다.

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting, AutoPipelineForImage2Image
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16, variant="fp16"
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 기본 이미지와 마스크 이미지 로드
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png")

prompt = "concept art digital painting of an elven castle, inspired by lord of the rings, highly detailed, 8k"
image_inpainting = pipeline(prompt=prompt, image=init_image, mask_image=mask_image).images[0]

# SDXL용으로 이미지 크기 조정 (1024x1024)
image_inpainting = image_inpainting.resize((1024, 1024))

이제 이미지를 SDXL의 리파이너 모델을 사용하여 다른 인페인팅 파이프라인에 전달하여 이미지 세부 정보와 품질을 향상시킵니다.

In [ ]:
pipeline = AutoPipelineForInpainting.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0", torch_dtype=torch.float16, variant="fp16"
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

image = pipeline(prompt=prompt, image=image_inpainting, mask_image=mask_image, output_type="latent").images[0]

<Tip>

불필요한 디코딩-인코딩 단계를 피하기 위해 파이프라인에서 `output_type="latent"`를 지정하여 모든 출력을 잠재 공간에 유지하는 것이 중요합니다. 이는 연결된 파이프라인이 동일한 VAE를 사용하는 경우에만 작동합니다. 예를 들어 [텍스트-이미지-인페인트](#text-to-image-to-inpaint) 섹션에서 Kandinsky 2.2는 Stable Diffusion 모델과 다른 VAE 클래스를 사용하므로 작동하지 않습니다. 하지만 두 파이프라인 모두에 Stable Diffusion v1.5를 사용하는 경우 둘 다 [AutoencoderKL](https://huggingface.co/docs/diffusers/main/en/api/models/autoencoderkl#diffusers.AutoencoderKL)을 사용하므로 모든 것을 잠재 공간에 유지할 수 있습니다.

</Tip>

마지막으로 이 이미지를 이미지-이미지 파이프라인에 전달하여 마무리 작업을 수행할 수 있습니다. 기존 파이프라인 구성 요소를 재사용하고 모든 파이프라인 구성 요소를 불필요하게 메모리에 다시 로드하는 것을 방지하려면 [from_pipe()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/auto_pipeline#diffusers.AutoPipelineForImage2Image.from_pipe) 메서드를 사용하는 것이 더 효율적입니다.

In [ ]:
pipeline = AutoPipelineForImage2Image.from_pipe(pipeline)
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

image = pipeline(prompt=prompt, image=image).images[0]
make_image_grid([init_image, mask_image, image_inpainting, image], rows=2, cols=2)

<div class="flex flex-row gap-4">
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">초기 이미지</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-to-image-chain.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">인페인트</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-to-image-final.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">이미지-이미지 변환</figcaption>
  </div>
</div>

이미지-이미지 변환과 인페인팅은 실제로 매우 유사한 작업입니다. 이미지-이미지 변환은 제공된 기존 이미지와 유사한 새 이미지를 생성합니다. 인페인팅은 동일한 작업을 수행하지만 마스크로 정의된 이미지 영역만 변환하고 이미지의 나머지 부분은 변경되지 않습니다. 인페인팅은 특정 변경 사항을 적용하기 위한 보다 정밀한 도구로 생각할 수 있으며, 이미지-이미지 변환은 보다 광범위한 변경 사항을 적용하기 위한 더 넓은 범위를 갖습니다.

## 이미지 생성 제어

원하는 대로 정확하게 보이는 이미지를 생성하는 것은 어려울 수 있으며, 이것이 바로 제어된 생성 기술과 모델이 유용한 이유입니다. `negative_prompt`와 같은 매개변수를 구성하여 생성의 특정 측면을 제어할 수 있지만 이미지 생성을 제어하는 더 좋고 효율적인 방법이 있습니다.

### 프롬프트 가중치

프롬프트 가중치는 프롬프트에서 개념의 표현을 정량화 가능한 방식으로 조정하는 방법을 제공합니다. 이를 사용하여 프롬프트에서 각 개념에 대한 텍스트 임베딩 벡터의 크기를 늘리거나 줄일 수 있으며, 이는 결과적으로 각 개념이 얼마나 생성되는지를 결정합니다. [Compel](https://github.com/damian0815/compel) 라이브러리는 프롬프트 가중치를 조정하고 임베딩을 생성하기 위한 직관적인 구문을 제공합니다. [프롬프트 가중치](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../using-diffusers/weighted_prompts) 가이드에서 임베딩을 만드는 방법을 알아보세요.

임베딩을 생성한 후에는 [AutoPipelineForInpainting](https://huggingface.co/docs/diffusers/main/en/api/pipelines/auto_pipeline#diffusers.AutoPipelineForInpainting)의 `prompt_embeds`(부정 프롬프트를 사용하는 경우 `negative_prompt_embeds`) 매개변수에 전달합니다. 임베딩은 `prompt` 매개변수를 대체합니다.

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import make_image_grid

pipeline = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16,
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

image = pipeline(prompt_embeds=prompt_embeds, # Compel에서 생성됨
    negative_prompt_embeds=negative_prompt_embeds, # Compel에서 생성됨
    image=init_image,
    mask_image=mask_image
).images[0]
make_image_grid([init_image, mask_image, image], rows=1, cols=3)

### ControlNet

ControlNet 모델은 Stable Diffusion과 같은 다른 확산 모델과 함께 사용되며, 이미지가 생성되는 방식을 제어하는 훨씬 더 유연하고 정확한 방법을 제공합니다. ControlNet은 확산 모델이 해당 특징을 보존하도록 안내하는 추가 조건 이미지 입력을 허용합니다.

예를 들어, 인페인트 이미지에 사전 학습된 ControlNet으로 이미지를 조건화해 보겠습니다.

In [ ]:
import torch
import numpy as np
from diffusers import ControlNetModel, StableDiffusionControlNetInpaintPipeline
from diffusers.utils import load_image, make_image_grid

# ControlNet 로드
controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_inpaint", torch_dtype=torch.float16, variant="fp16")

# 파이프라인에 ControlNet 전달
pipeline = StableDiffusionControlNetInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting", controlnet=controlnet, torch_dtype=torch.float16, variant="fp16"
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

# 기본 이미지와 마스크 이미지 로드
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png")

# 제어 이미지 준비
def make_inpaint_condition(init_image, mask_image):
    init_image = np.array(init_image.convert("RGB")).astype(np.float32) / 255.0
    mask_image = np.array(mask_image.convert("L")).astype(np.float32) / 255.0

    assert init_image.shape[0:1] == mask_image.shape[0:1], "image and image_mask must have the same image size"
    init_image[mask_image > 0.5] = -1.0  # 마스크된 픽셀로 설정
    init_image = np.expand_dims(init_image, 0).transpose(0, 3, 1, 2)
    init_image = torch.from_numpy(init_image)
    return init_image

control_image = make_inpaint_condition(init_image, mask_image)

이제 기본 이미지, 마스크 및 제어 이미지에서 이미지를 생성합니다. 생성된 이미지에서 기본 이미지의 특징이 강력하게 보존되는 것을 알 수 있습니다.

In [ ]:
prompt = "concept art digital painting of an elven castle, inspired by lord of the rings, highly detailed, 8k"
image = pipeline(prompt=prompt, image=init_image, mask_image=mask_image, control_image=control_image).images[0]
make_image_grid([init_image, mask_image, PIL.Image.fromarray(np.uint8(control_image[0][0])).convert('RGB'), image], rows=2, cols=2)

이미지-이미지 파이프라인에 연결하여 새로운 [스타일](https://huggingface.co/nitrosocke/elden-ring-diffusion)을 적용하여 한 단계 더 나아갈 수 있습니다.

In [ ]:
from diffusers import AutoPipelineForImage2Image

pipeline = AutoPipelineForImage2Image.from_pretrained(
    "nitrosocke/elden-ring-diffusion", torch_dtype=torch.float16,
)
pipeline.enable_model_cpu_offload()
# xFormers가 설치되어 있지 않거나 PyTorch 2.0 이상이 설치되어 있는 경우 다음 줄을 제거합니다.
pipeline.enable_xformers_memory_efficient_attention()

prompt = "elden ring style castle" # 프롬프트에 "elden ring style" 토큰 포함
negative_prompt = "bad architecture, deformed, disfigured, poor details"

image_elden_ring = pipeline(prompt, negative_prompt=negative_prompt, image=image).images[0]
make_image_grid([init_image, mask_image, image, image_elden_ring], rows=2, cols=2)

<div class="flex flex-row gap-4">
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">초기 이미지</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-controlnet.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">ControlNet 인페인트</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint-img2img.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">이미지-이미지 변환</figcaption>
  </div>
</div>

## 최적화

리소스가 제한적인 경우 확산 모델을 실행하는 것이 어렵고 느릴 수 있지만 몇 가지 최적화 트릭을 사용하면 그렇지 않습니다. 활성화할 수 있는 가장 크고 쉬운 최적화 중 하나는 메모리 효율적인 어텐션으로 전환하는 것입니다. PyTorch 2.0을 사용하는 경우 [스케일드 닷 프로덕트 어텐션](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/fp16#scaled-dot-product-attention)이 자동으로 활성화되므로 다른 작업을 수행할 필요가 없습니다. PyTorch 2.0 사용자가 아닌 경우 [xFormers](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/xformers)의 메모리 효율적인 어텐션 구현을 설치하여 사용할 수 있습니다. 두 옵션 모두 메모리 사용량을 줄이고 추론 속도를 높입니다.

모델을 CPU로 오프로드하여 메모리를 더욱 절약할 수도 있습니다.

```diff
+ pipeline.enable_xformers_memory_efficient_attention()
+ pipeline.enable_model_cpu_offload()
```

추론 코드를 더욱 빠르게 실행하려면 [`torch_compile`](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/fp16#torchcompile)을 사용합니다. 일반적으로 파이프라인에서 가장 집약적인 구성 요소인 UNet 주위에 `torch.compile`을 래핑해야 합니다.

In [ ]:
pipeline.unet = torch.compile(pipeline.unet, mode="reduce-overhead", fullgraph=True)

자세한 내용은 [메모리 사용량 줄이기](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/memory) 및 [추론 가속화](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../optimization/fp16) 가이드를 참조하세요.